In [1]:
import pandas as pd

In [2]:
import fetch_gr as fgr

In [3]:
df_cases = fgr.get_canton_data_df()
df_cases

,Datum,Region,Neue_Faelle
0,2020-02-26,Albula,0
1,2020-02-26,Bernina,0
2,2020-02-26,Engiadina Bassa/Val Müstair,0
3,2020-02-26,Imboden,0
4,2020-02-26,Landquart,0
...,...,...,...
1995,2020-08-10,Imboden,0
1996,2020-08-10,Landquart,0
1997,2020-08-10,Maloja,1
1998,2020-08-10,Moesa,0


In [4]:
import fetch_municipalities as fmp

In [5]:
df_municipalities = fmp.get_municipalities_df()
df_municipalities

,Gemeindename,Bezirksname,Kanton
BFS_Nr,,,
3542,Albula/Alvra,Albula,GR
3701,Andeer,Viamala,GR
3921,Arosa,Plessur,GR
3681,Avers,Viamala,GR
3781,Bever,Maloja,GR
...,...,...,...
3714,Rheinwald,Viamala,GR
3785,La Punt Chamues-ch,Maloja,GR
3901,Chur,Plessur,GR


In [6]:
import fetch_cantons_metadata as fcm

In [7]:
df_population_for_bfsNr = fcm.get_cantons_metadata_df()
df_population_for_bfsNr

,Gemeindename,Einwohner,Gesamtflaeche_in_km2
BFS_Nr,,,
1,Aeugst am Albis,1982,7.91
2,Affoltern am Albis,12229,10.59
3,Bonstetten,5548,7.43
4,Hausen am Albis,3701,13.6
5,Hedingen,3734,6.53
...,...,...,...
6806,Vendlincourt,540,9.15
6807,Basse-Allaine,1230,23.04
6808,Clos du Doubs,1286,61.75


In [8]:
df_all = pd.merge(df_population_for_bfsNr, df_municipalities , how='inner', left_on='BFS_Nr', right_on='BFS_Nr')
# df_all = df.loc[:,~df.columns.duplicated()]
df_all.rename(columns={'Gemeindename_y': 'Gemeindename'}, inplace=True)
df_all.drop('Gemeindename_x', axis=1, inplace=True)
df_all

,Einwohner,Gesamtflaeche_in_km2,Gemeindename,Bezirksname,Kanton
BFS_Nr,,,,,
3506,2780,42.51,Vaz/Obervaz,Albula,GR
3513,535,21.81,Lantsch/Lenz,Albula,GR
3514,234,11.35,Schmitten (GR),Albula,GR
3542,1310,93.93,Albula/Alvra,Albula,GR
3543,2356,323.77,Surses,Albula,GR
...,...,...,...,...,...
3983,362,136.22,Medel (Lucmagn),Surselva,GR
3985,1158,101.88,Sumvitg,Surselva,GR
3986,1238,133.91,Tujetsch,Surselva,GR


## Mapping von Region auf Bezirksnamen

In [9]:
set_bezirksnamen = set(sorted(df_municipalities['Bezirksname'])) # From Municipality Stammdaten
print(set_bezirksnamen)
print(len(set_bezirksnamen))

{'Landquart', 'Prättigau / Davos', 'Engiadina B./Val Müstair', 'Albula', 'Bernina', 'Imboden', 'Moesa', 'Surselva', 'Maloja', 'Plessur', 'Viamala'}
11


In [10]:
set_regionen = set(sorted(df_cases['Region'])) # From Cases, i.e. the region a case belongs to
print(set_regionen)
print(len(set_regionen))

{'Engiadina Bassa/Val Müstair', 'Landquart', 'ohne Wohnsitz', 'Albula', 'Bernina', 'Imboden', 'Moesa', 'Surselva', 'Maloja', 'Plessur', 'Prättigau/Davos', 'Viamala'}
12


### Was machen wir mit 'ohne Wohnsitz'? 
Um die Kantonszahlen nicht zu verfälschen, müssten sie evtl. auch verteilt werden auf die Bezirke...

In [11]:
# Region (from Cases): Bezirksname (from Municipalities)
dict_bezirks_mapping = {'Albula': 'Albula',
 'Bernina': 'Bernina',
 'Engiadina Bassa/Val Müstair': 'Engiadina B./Val Müstair',
 'Imboden': 'Imboden',
 'Landquart': 'Landquart',
 'Maloja': 'Maloja',
 'Moesa': 'Moesa',
 'Plessur': 'Plessur',
 'Prättigau/Davos': 'Prättigau / Davos',
 'Surselva': 'Surselva',
 'Viamala': 'Viamala'}

Neue Spalte 'Bezirksname' welche anhand 'Region' gemapped wurde:

In [13]:
df_cases['Bezirksname'] = df_cases['Region'].apply(lambda region: dict_bezirks_mapping.get(region))
df_cases

,Datum,Region,Neue_Faelle,Bezirksname
0,2020-02-26,Albula,0,Albula
1,2020-02-26,Bernina,0,Bernina
2,2020-02-26,Engiadina Bassa/Val Müstair,0,Engiadina B./Val Müstair
3,2020-02-26,Imboden,0,Imboden
4,2020-02-26,Landquart,0,Landquart
...,...,...,...,...
1995,2020-08-10,Imboden,0,Imboden
1996,2020-08-10,Landquart,0,Landquart
1997,2020-08-10,Maloja,1,Maloja
1998,2020-08-10,Moesa,0,Moesa


 Nicht gefundene Einträge (z.B. 'ohne Wohnsitz') werden mit None abgefüllt:

In [14]:
display(df_cases[df_cases['Bezirksname'].isnull()])
print(df_cases[df_cases['Bezirksname'].isnull()].sum())

,Datum,Region,Neue_Faelle,Bezirksname
11,2020-02-26,ohne Wohnsitz,2,None
23,2020-02-27,ohne Wohnsitz,0,None
35,2020-02-28,ohne Wohnsitz,4,None
47,2020-02-29,ohne Wohnsitz,0,None
59,2020-03-01,ohne Wohnsitz,0,None
...,...,...,...,...
1943,2020-08-05,ohne Wohnsitz,0,None
1955,2020-08-06,ohne Wohnsitz,1,None
1967,2020-08-07,ohne Wohnsitz,0,None
1979,2020-08-08,ohne Wohnsitz,1,None


Sum:  Neue_Faelle    67
Bezirksname     0
dtype: object


## Marcos ex-Müll

In [22]:
df_all['BFS_Nr'] = df_all.index

In [23]:
df_municipality_cases = pd.merge(df_all, df_cases, how='inner', left_on='Bezirksname', right_on='Bezirksname')
df_municipality_cases

,Einwohner,Gesamtflaeche_in_km2,Gemeindename,Bezirksname,Kanton,BFS_Nr,Datum,Region,Neue_Faelle
0,2780,42.51,Vaz/Obervaz,Albula,GR,3506,2020-02-26,Albula,0
1,2780,42.51,Vaz/Obervaz,Albula,GR,3506,2020-02-27,Albula,0
2,2780,42.51,Vaz/Obervaz,Albula,GR,3506,2020-02-28,Albula,0
3,2780,42.51,Vaz/Obervaz,Albula,GR,3506,2020-02-29,Albula,0
4,2780,42.51,Vaz/Obervaz,Albula,GR,3506,2020-03-01,Albula,0
...,...,...,...,...,...,...,...,...,...
16651,8889,18.86,Landquart,Landquart,GR,3955,2020-08-06,Landquart,0
16652,8889,18.86,Landquart,Landquart,GR,3955,2020-08-07,Landquart,0
16653,8889,18.86,Landquart,Landquart,GR,3955,2020-08-08,Landquart,0
16654,8889,18.86,Landquart,Landquart,GR,3955,2020-08-09,Landquart,0
